In [28]:
from typing import Dict

In [29]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer
import joblib
import mlflow

In [30]:
def rmsle(y_true, y_pred) -> float:
    pass

def rmse(y_true, y_pred) -> float:
    return np.sqrt(mean_squared_error(y_true, y_pred))

rmse_score = make_scorer(rmse, greater_is_better=False)

In [31]:
def add_key_prefix(d: Dict, prefix = 'best_') -> Dict:
    return {prefix + key: value for key, value in d.items()}

In [32]:
def grid_search(ds: np.ndarray):
    
    y = np.log1p(ds[:, 0])
    x = ds[:, 1:]
    
    x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2)
    
    param_grid = dict(
        n_estimators=[20, 40, 60, 80, 100],
        max_depth=[4, 8, 12, None],
        max_features=['auto', 'sqrt'],
    )
        
    mlflow.set_experiment('baseline')
    with mlflow.start_run() as run:
        
        mlflow.log_params(param_grid)
        
        regressor = GridSearchCV(
            RandomForestRegressor(n_jobs=-1),
            param_grid=param_grid,
            cv=3,
            scoring=rmse_score,
            verbose=2,
            refit=True,
        )

        regressor.fit(x_train, y_train)
        
        best_model = regressor.best_estimator_
        best_param = add_key_prefix(regressor.best_params_)
        best_rmse = - regressor.best_score_

        joblib.dump(best_model, 'out/model.sav')
        
        y_pred_val = best_model.predict(x_val)
        rmse_val = rmse(y_val, y_pred_val)

        mlflow.log_params(best_param)
        mlflow.log_metrics(dict(
            rmse=best_rmse,
            rmse_val=rmse_val,
        ))
        mlflow.log_artifact('./out/model.sav')
        mlflow.end_run()

In [33]:
def holdout(ds: np.ndarray):
    y = np.log1p(ds[:, 0])
    x = ds[:, 1:]
    
    x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2)
    
    params = dict(
        n_estimators=10,
        max_depth=None,
        max_features='auto',
    )
        
    mlflow.set_experiment('baseline')
    with mlflow.start_run() as run:
        
        mlflow.log_params(params)
        
        model = RandomForestRegressor(**params, verbose=2, n_jobs=-1)

        model.fit(x_train, y_train)
        
        joblib.dump(model, 'out/model.sav')
        
        y_pred_train = model.predict(x_train)
        y_pred_val = model.predict(x_val)
        rmse_train = rmse(y_train, y_pred_train)
        rmse_val = rmse(y_val, y_pred_val)

        mlflow.log_metrics(dict(
            rmse_train=rmse_train,
            rmse_val=rmse_val,
        ))
        mlflow.log_artifact('./out/model.sav')
        mlflow.end_run()

In [34]:
dataset_train = np.load('dataset_train.npy')

In [15]:
def sample(a: np.ndarray) -> np.ndarray:
    return a[np.random.choice(a.shape[0], a.shape[0] // 100, replace=False), :]

In [ ]:
grid_search(dataset_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV] max_depth=4, n_estimators=20, max_features=auto .................
building tree 1 of 20
building tree 2 of 20
building tree 3 of 20
building tree 4 of 20
building tree 5 of 20
building tree 6 of 20
building tree 7 of 20
building tree 8 of 20
building tree 9 of 20
building tree 10 of 20
building tree 11 of 20
building tree 12 of 20
building tree 13 of 20
building tree 14 of 20
building tree 15 of 20
building tree 16 of 20
building tree 17 of 20
building tree 18 of 20
building tree 19 of 20
building tree 20 of 20


[Parallel(n_jobs=-1)]: Done  11 out of  20 | elapsed:  2.9min remaining:  2.4min
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  4.5min finished
[Parallel(n_jobs=16)]: Done  11 out of  20 | elapsed:    0.3s remaining:    0.2s
[Parallel(n_jobs=16)]: Done  20 out of  20 | elapsed:    0.6s finished
[Parallel(n_jobs=16)]: Done  11 out of  20 | elapsed:    1.3s remaining:    1.0s
[Parallel(n_jobs=16)]: Done  20 out of  20 | elapsed:    2.2s finished


[CV] .. max_depth=4, n_estimators=20, max_features=auto, total= 4.6min
[CV] max_depth=4, n_estimators=20, max_features=auto .................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.7min remaining:    0.0s


building tree 1 of 20
building tree 2 of 20
building tree 3 of 20
building tree 4 of 20
building tree 5 of 20
building tree 6 of 20
building tree 7 of 20
building tree 8 of 20
building tree 9 of 20
building tree 10 of 20
building tree 11 of 20
building tree 12 of 20
building tree 13 of 20
building tree 14 of 20
building tree 15 of 20
building tree 16 of 20
building tree 17 of 20
building tree 18 of 20
building tree 19 of 20
building tree 20 of 20


...

building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 40.4min finished
[Parallel(n_jobs=16)]: Done   9 tasks      | elapsed:    0.5s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    4.8s finished
[Parallel(n_jobs=16)]: Done   9 tasks      | elapsed:    1.6s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:   14.4s finished


[CV] . max_depth=8, n_estimators=100, max_features=auto, total=40.6min
[CV] max_depth=8, n_estimators=100, max_features=auto ................
building tree 1 of 100building tree 2 of 100building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100

building tree 9 of 100
building tree 10 of 100

building tree 11 of 100
building tree 12 of 100building tree 13 of 100
building tree 14 of 100

building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100


[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  6.3min


building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
